# Lezione 4 - Generazione di numeri pseudo-casuali
## Generatore lineare congruenziale
$$ x_{n+1} = (A \cdot x_{n} + C) \mod M $$

con 
$$ M > 0 $$ 
$$ 0 < A < M $$ 
$$ 0 < x_{0} < M $$
$$ M \sim 10^{32} $$

Si tratta di un esempio di formula ricorsiva per calcolare il numero successivo in una sequenza di numeri pseudo-casuali dato un numero della sequenza. E' un esempio storico, esistono ovviamente generatori pseudo-random più sofisticati.

## Libreria random
La libreria `random` di Python contiene un generatore di numeri pseudo-casuali `random()`

In [7]:
import random

rand_list: list[float] = []

for i in range(5):
    rand_list.append(random.random())
    print(f"{i+1} numero: {rand_list[-1]}")

1 numero: 0.10636209341028924
2 numero: 0.8305194858702567
3 numero: 0.17082047801572575
4 numero: 0.22851870040785782
5 numero: 0.16892373273349592


Il metodo `random.random()` genera numeri pseudo-casuali distribuiti uniformemente nell'intervallo $ [0, 1] $. Il seed usato di default si basa sul tempo del sistema operativo. Per impostare il seed, si usa
``` Python
random.seed(seed)
```
E' importante poter riprodurre la stessa sequenza di numeri pseudo-random per questioni legate al testing. A meno che ci siano motivi validi per fare altrimenti, il seed viene inizializzato solo una volta all'interno del programma. E' anche possibile generare numeri pseudo-casuali interi, utilizzando il metodo `random.randint()`

In [18]:
# Simulazione del lancio di un dado
rand_list_int: list[int] = []

for i in range(5):
    rand_list_int.append(random.randint(1, 6))
    print(f"{i+1} numero: {rand_list_int[-1]}")

1 numero: 1
2 numero: 4
3 numero: 3
4 numero: 4
5 numero: 5


## Generazione di numeri pseudo-casuali distribuiti uniformemente con la libreria random
Si utilizza il metodo `rand_range()`, definito come segue

In [37]:
def rand_range(x_min: float, x_max: float) -> float:
    """ Genera un numero pseudo-casuale distribuito uniformemente nell'intervallo di estremi x_min e x_max """

    return x_min + ((x_max - x_min) * random.random())

# Esempio
x_min = 5.0
x_max = 10.0

num = rand_range(x_min, x_max)
print(f"Numero tra {x_min} e {x_max}: {num}")

Numero tra 5 e 10: 7.776136206077283


## Metodo Try-And-Catch (TAC)
Il metodo TAC viene utilizzato per generare sequenze di numeri pseudo-casuali distribuiti secondo una qualunque PDF, a partire da due sequenze di numeri pseudo-casuali distribuiti uniformemente. L'idea è quella di generare numeri pseudo-casuali in modo proporzionale all'area sottesa dalla PDF, cioè il suo integrale (che rappresenta una probabilità). Si popola il piano con coppie di numeri pseudo-casuali distribuiti uniformemente $ (x, y) $, generati utilizzando la funzione `rand_range()` e si aggiunge $ x $ alla sequenza solo se $ y < f(x) $, cioè solo se $ x $ cade all'interno dell'area della PDF.

In [56]:
def rand_TAC(f, x_min: float, x_max: float, y_max: float) -> float:
    """ Genera un numero pseudo-casuale distribuito secondo la PDF "f" nell'intervallo di estremi x_min e x_max """

    x = rand_range(x_min, x_max)
    y = rand_range(0, y_max)

    while y > f(x):
        x = rand_range(x_min, x_max)
        y = rand_range(0, y_max)

    return x

# Esempio: numero casuale con distribuzione gaussiana
from scipy.stats import norm

media = 0.0
sigma = 0.5
gauss = norm(media, sigma)
x_coord = np.linspace(-100, 100, 1000000)

gauss_num = rand_TAC(gauss.pdf, -sigma, sigma, gauss.pdf(0))
print(f"Numero gaussiano: {gauss_num}")

Numero gaussiano: -0.22616452970680312


## Metodo della funzione inversa
Conoscendo la forma analitica di una PDF $ f(x) $ e la forma analitica della sua primitiva $ F(x) $, cioè la CDF, è possibile generare numeri pseudo-casuali distribuiti secondo $ f(x) $ a partire da numeri pseudo-casuali con distribuzione uniforme. Tuttavia, questo è possibile solo se la cumulativa $ F(x) $ è invertibile. I passaggi sono i seguenti:

- Si determina la forma funzionale di $ F(x) $
- Si determina la forma funzionale dell'inversa $ x = F^{-1}(y) $ e si usa $ x $ come numero pseudo-random distribuito secondo $ f(x) $

Si osserva che questo è possibile perché dove $ f(x) $ è più alta, si ha che $ F(x) $ è più pendente (essendo $ f $ la derivata di $ F $), perciò il numero di numeri pseudo-casuali generati in un intervallo $ \Delta y $ risulta proporzionale all'area sottesa dalla curva $ f(x) $ sopra l'intervallo di ampiezza $ \Delta x $.

## Teorema centrale del limite
- Si parte da una sequenza di N numeri casuali distribuiti uniformemente
- Si calcola la media $ \bar{x} $, che sarà distribuita secondo una distribuzione che approssima una gaussiana sempre meglio al crescere di N
- Per $ N \to \infty $ si ottiene una distribuzione gaussiana